# 필수 실행

## 구글 드라이브 연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 필수 설치

In [ ]:
!pip install category_encoders

## 필수 임포트

In [ ]:
import os
import random
import numpy as np
import pandas as pd

# encoder
from sklearn.preprocessing import LabelEncoder
from category_encoders.target_encoder import TargetEncoder

# 그래프
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import matplotlib.font_manager as fm

# progressing bar
import tqdm

# Regressor Model
from sklearn.linear_model import LinearRegression,Ridge,Lasso,RidgeCV
from sklearn.ensemble import RandomForestRegressor,BaggingRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from lightgbm.sklearn import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import StackingRegressor

# 평가지표
from sklearn.metrics import mean_squared_log_error

# GridSearch
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_regression
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split

pd.set_option('display.max_columns', None)

# seed 고정
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42)

import warnings
warnings.filterwarnings('ignore')

## csv파일 불러오기

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/daegu/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/daegu/test.csv')
submission_df = pd.read_csv('/content/drive/MyDrive/daegu/sample_submission.csv')
extra_train_df = pd.read_csv('/content/drive/MyDrive/daegu/external_open/countrywide_accident.csv')
light_df = pd.read_csv('/content/drive/MyDrive/daegu/external_open/대구보안등정보.csv',encoding='cp949')
child_area_df = pd.read_csv('/content/drive/MyDrive/daegu/external_open/대구어린이보호구역정보.csv',encoding='cp949')
parking_df = pd.read_csv('/content/drive/MyDrive/daegu/external_open/대구주차장정보.csv',encoding='cp949')
cctv_df = pd.read_csv('/content/drive/MyDrive/daegu/external_open/대구CCTV정보.csv',encoding='cp949')

## 평가지표 함수

In [ ]:
def rmsle(y_true, y_pred):
    return np.sqrt(np.mean(np.square(np.log1p(y_pred) - np.log1p(y_true))))

# RMSLE를 평가 지표로 변환
rmsle_scorer = make_scorer(rmsle, greater_is_better=False)

# EDA

## train/test data 합치기 및 컬럼정리


### 추가적인 data에서 필요없다고 생각하는 컬럼정리

In [ ]:
light_df = light_df.drop(columns=['보안등위치명','소재지도로명주소','위도','경도'])
child_area_df = child_area_df.drop(columns=['소재지도로명주소','위도','경도','데이터기준일자'])
parking_df = parking_df.drop(columns=['주차장관리번호','소재지도로명주소','위도','경도','데이터기준일자'])
cctv_df = cctv_df.drop(columns=['소재지도로명주소','위도','경도'])

### 보안등 data
- 보안등의 설치개수 동마다의 총합/평균 두 종류로 했다.
- 설치형태는 one-hot encoding으로 각각 동마다의 총합으로 했다.

In [ ]:
location_pattern = r'(\S+) (\S+) (\S+) (\S+)'

In [ ]:
light_df[['도시', '구', '동', '번지']] = light_df['소재지지번주소'].str.extract(location_pattern)
light_df = light_df.drop(columns=['소재지지번주소', '번지'])

In [ ]:
light_df = pd.get_dummies(light_df, columns=['설치형태'])

In [ ]:
light_df['설치개수평균'] = light_df['설치개수']

light_ = light_df[['설치형태_건축물', '설치형태_전용주', '설치형태_한전주',
                   '설치개수','도시', '구', '동']].groupby(['도시', '구', '동']).sum().reset_index()
light__ = light_df[['설치개수평균','도시', '구', '동']].groupby(['도시', '구', '동']).mean().round(2).reset_index()

In [ ]:
light_.reset_index(inplace=True, drop=True)
light__.reset_index(inplace=True, drop=True)

### 어린이 보호구역 data
- 어린이 보호구역에서 동마다 CCTV설치개수의 총합/평균을 했다.
- 시설종류, 관할경찰서, CCTV설치여부를 one-hot encoding을 하여서 각각 동마다의 총합으로 했다.

In [1]:
child_area_df['CCTV설치대수'].fillna(0, inplace=True)
child_area_df['CCTV설치한곳의수'] = 1
child_area_df['CCTV설치대수평균'] = child_area_df['CCTV설치대수']

NameError: ignored

In [ ]:
child_area_df[['도시', '구', '동', '번지']] = child_area_df['소재지지번주소'].str.extract(location_pattern)
child_area_df = child_area_df.drop(columns=['소재지지번주소', '번지'])

In [ ]:
child_area_df = pd.get_dummies(child_area_df, columns=['시설종류','관할경찰서명','CCTV설치여부'])

In [ ]:
child_area_ = child_area_df[['시설종류_어린이집',
       '시설종류_유치원', '시설종류_초등학교', '시설종류_특수학교', '시설종류_학원', '관할경찰서명_강북경찰서',
       '관할경찰서명_남부경찰서', '관할경찰서명_달성경찰서', '관할경찰서명_대구광역시 중부경찰서', '관할경찰서명_대구동부경찰서',
       '관할경찰서명_북부경찰서', '관할경찰서명_서부경찰서', '관할경찰서명_수성경찰서', 'CCTV설치여부_N',
       'CCTV설치여부_Y','CCTV설치한곳의수','CCTV설치대수','도시', '구', '동']].groupby(['도시', '구', '동']).sum().reset_index()
child_area__ = child_area_df[['CCTV설치대수평균','도시', '구', '동']].groupby(['도시', '구', '동']).mean().round(2).reset_index()

In [ ]:
child_area_.reset_index(inplace=True, drop=True)
child_area__.reset_index(inplace=True, drop=True)

### 주차장 data
- 주차구획수 동마다의 총합/평균으로 했다.
- 급지구분, 결제방법, 주차장구분, 주차장유형은 one-hot encoding으로 해서 동마다의 총합으로 했다.

In [ ]:
parking_df[['도시', '구', '동', '번지']] = parking_df['소재지지번주소'].str.extract(location_pattern)
parking_df = parking_df.drop(columns=['소재지지번주소', '번지'])

In [ ]:
parking_df['주차구획수평균'] = parking_df['주차구획수']

In [ ]:
parking_df = pd.get_dummies(parking_df, columns=['급지구분','결제방법','주차장구분','주차장유형'])

In [ ]:
parking_df_ = parking_df[['급지구분_1', '급지구분_2', '급지구분_3', '결제방법_교통카드', '결제방법_무료',
       '결제방법_신용카드', '결제방법_신용카드, 교통카드', '결제방법_카드', '결제방법_현금', '결제방법_현금+신용카드',
       '결제방법_현금, 신용카드', '결제방법_현금,카드', '주차장구분_공영', '주차장구분_민영', '주차장유형_노상',
       '주차장유형_노외','주차구획수','도시', '구', '동']].groupby(['도시', '구', '동']).sum().reset_index()
parking_df__ = parking_df[['주차구획수평균','주차기본요금','추가단위요금',
                           '1일주차권요금','월정기권요금','도시', '구', '동']].groupby(['도시', '구', '동']).mean().round(2).reset_index()

In [ ]:
parking_df_.reset_index(inplace=True, drop=True)
parking_df__.reset_index(inplace=True, drop=True)

### CCTV data
- 무인CCTV를 동마다의 설치한 곳의 총합으로 했다.
- 설치연도와 제한속도는 동마다의 평균으로 구해줬다.
- 도로 노선방향은 one-hot encoding으로 해서 동마다의 총합으로 해줬다.

In [ ]:
cctv_df['무인CCTV설치한곳의수'] = 1

In [ ]:
cctv_df[['도시', '구', '동', '번지']] = cctv_df['소재지지번주소'].str.extract(location_pattern)
cctv_df = cctv_df.drop(columns=['소재지지번주소', '번지'])

In [ ]:
cctv_df = pd.get_dummies(cctv_df, columns=['도로노선방향'])

In [ ]:
cctv_ = cctv_df[['도로노선방향_1', '도로노선방향_2','도로노선방향_3',
                 '무인CCTV설치한곳의수','도시', '구', '동']].groupby(['도시', '구', '동']).sum().reset_index()
cctv__ = cctv_df[['설치연도','제한속도','도시', '구', '동']].groupby(['도시', '구', '동']).mean().round(2).reset_index()

In [ ]:
cctv_.reset_index(inplace=True, drop=True)
cctv__.reset_index(inplace=True, drop=True)

### '사고일시' 분리

In [ ]:
time_pattern = r'(\d{4})-(\d{1,2})-(\d{1,2}) (\d{1,2})'

train_df[['연', '월', '일', '시간']] = train_df['사고일시'].str.extract(time_pattern)
train_df[['연', '월', '일', '시간']] = train_df[['연', '월', '일', '시간']].apply(pd.to_numeric)
train_df = train_df.drop(columns=['사고일시'])

test_df[['연', '월', '일', '시간']] = test_df['사고일시'].str.extract(time_pattern)
test_df[['연', '월', '일', '시간']] = test_df[['연', '월', '일', '시간']].apply(pd.to_numeric)
test_df = test_df.drop(columns=['사고일시'])

### '시군구' 분리

In [ ]:
location_pattern = r'(\S+) (\S+) (\S+)'

train_df[['도시', '구', '동']] = train_df['시군구'].str.extract(location_pattern)
train_df = train_df.drop(columns=['시군구'])

test_df[['도시', '구', '동']] = test_df['시군구'].str.extract(location_pattern)
test_df = test_df.drop(columns=['시군구'])

### '도로형태'분리

In [ ]:
road_pattern = r'(.+) - (.+)'

train_df[['도로형태1', '도로형태2']] = train_df['도로형태'].str.extract(road_pattern)

test_df[['도로형태1', '도로형태2']] = test_df['도로형태'].str.extract(road_pattern)

### 모든 데이터 ['도시','구','동']기준으로 merge하기

In [ ]:
train_df = train_df.drop(columns=['사고유형 - 세부분류', '법규위반',
                                  '가해운전자 차종', '가해운전자 성별','가해운전자 연령', '가해운전자 상해정도',
                                  '피해운전자 차종','피해운전자 성별', '피해운전자 연령', '피해운전자 상해정도',
                                  '중상자수', '경상자수', '부상자수',])

In [ ]:
train_df = pd.merge(train_df, light_, how='left', on=['도시', '구', '동'])
train_df = pd.merge(train_df, light__, how='left', on=['도시', '구', '동'])

test_df = pd.merge(test_df, light_, how='left', on=['도시', '구', '동'])
test_df = pd.merge(test_df, light__, how='left', on=['도시', '구', '동'])

In [ ]:
train_df = pd.merge(train_df, child_area_, how='left', on=['도시', '구', '동'])
train_df = pd.merge(train_df, child_area__, how='left', on=['도시', '구', '동'])

test_df = pd.merge(test_df, child_area_, how='left', on=['도시', '구', '동'])
test_df = pd.merge(test_df, child_area__, how='left', on=['도시', '구', '동'])

In [ ]:
train_df = pd.merge(train_df, parking_df_, how='left', on=['도시', '구', '동'])
train_df = pd.merge(train_df, parking_df__, how='left', on=['도시', '구', '동'])

test_df = pd.merge(test_df, parking_df_, how='left', on=['도시', '구', '동'])
test_df = pd.merge(test_df, parking_df__, how='left', on=['도시', '구', '동'])

In [ ]:
train_df = pd.merge(train_df, cctv_, how='left', on=['도시', '구', '동'])
train_df = pd.merge(train_df, cctv__, how='left', on=['도시', '구', '동'])

test_df = pd.merge(test_df, cctv_, how='left', on=['도시', '구', '동'])
test_df = pd.merge(test_df, cctv__, how='left', on=['도시', '구', '동'])

## 1. 데이터 이해

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39609 entries, 0 to 39608
Data columns (total 71 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  39609 non-null  object 
 1   요일                  39609 non-null  object 
 2   기상상태                39609 non-null  object 
 3   도로형태                39609 non-null  object 
 4   노면상태                39609 non-null  object 
 5   사고유형                39609 non-null  object 
 6   사망자수                39609 non-null  int64  
 7   중상자수                39609 non-null  int64  
 8   경상자수                39609 non-null  int64  
 9   부상자수                39609 non-null  int64  
 10  ECLO                39609 non-null  int64  
 11  연                   39609 non-null  int64  
 12  월                   39609 non-null  int64  
 13  일                   39609 non-null  int64  
 14  시간                  39609 non-null  int64  
 15  도시                  39609 non-null  object 
 16  구   

### 결측치가 전체 데이터의 20%이상인 컬럼들은 drop 해줬다.

In [ ]:
drop_lst = []

for i in train_df.columns.tolist():
  if train_df[i].isnull().sum() > 6600 :
    drop_lst.append(i)

train_df.drop(drop_lst, axis=1, inplace=True)

In [ ]:
train_df.describe()

,사망자수,중상자수,경상자수,부상자수,ECLO,연,월,일,시간,설치형태_건축물,설치형태_전용주,설치형태_한전주,설치개수,설치개수평균,시설종류_어린이집,시설종류_유치원,시설종류_초등학교,시설종류_특수학교,시설종류_학원,관할경찰서명_강북경찰서,관할경찰서명_남부경찰서,관할경찰서명_달성경찰서,관할경찰서명_대구광역시 중부경찰서,관할경찰서명_대구동부경찰서,관할경찰서명_북부경찰서,관할경찰서명_서부경찰서,관할경찰서명_수성경찰서,CCTV설치여부_N,CCTV설치여부_Y,CCTV설치한곳의수,CCTV설치대수,CCTV설치대수평균,급지구분_1,급지구분_2,급지구분_3,결제방법_교통카드,결제방법_무료,결제방법_신용카드,"결제방법_신용카드, 교통카드",결제방법_카드,결제방법_현금,결제방법_현금+신용카드,"결제방법_현금, 신용카드","결제방법_현금,카드",주차장구분_공영,주차장구분_민영,주차장유형_노상,주차장유형_노외,주차구획수,주차구획수평균,주차기본요금,추가단위요금,1일주차권요금,월정기권요금,도로노선방향_1,도로노선방향_2,도로노선방향_3,무인CCTV설치한곳의수,설치연도,제한속도
count,39609.000000,39609.000000,39609.000000,39609.000000,39609.000000,39609.000000,39609.000000,39609.000000,39609.000000,30096.000000,30096.000000,30096.000000,30096.000000,30096.000000,21183.000000,21183.000000,21183.000000,21183.000000,21183.000000,21183.000000,21183.000000,21183.000000,21183.000000,21183.0,21183.000000,21183.000000,21183.000000,21183.000000,21183.000000,21183.000000,21183.000000,21183.000000,33066.000000,33066.000000,33066.000000,33066.000000,33066.000000,33066.000000,33066.000000,33066.000000,33066.000000,33066.000000,33066.000000,33066.000000,33066.000000,33066.000000,33066.000000,33066.000000,33066.00000,33066.000000,33066.000000,24540.000000,23141.000000,22398.000000,38033.000000,38033.000000,38033.000000,38033.000000,38033.000000,38033.000000
mean,0.007776,0.262365,1.070085,0.126865,4.726704,2019.939332,6.678507,15.817087,13.934863,148.281067,8.084762,369.095827,881.939427,1.006913,3.263372,5.789831,4.001936,0.456168,0.042865,1.258226,2.435160,3.763490,0.179106,0.0,0.901336,2.093282,2.923571,3.593400,9.960770,13.554171,19.430109,1.813463,2.549961,4.501210,3.164217,0.009043,0.038166,0.020958,0.005141,0.061786,2.714752,0.000544,0.022319,0.083772,9.477893,0.737495,6.088762,4.126626,454.17202,52.584840,253.650388,421.779472,4910.845315,55673.601283,5.411800,4.442353,3.978913,13.833066,2017.872704,38.508138
std,0.090109,0.500845,0.992034,0.394670,3.207206,0.818317,3.404229,8.792314,5.626818,393.983807,16.747296,860.091743,1206.169820,0.013900,2.820451,6.873894,3.242962,1.378221,0.202556,3.366381,7.217335,12.910862,1.121680,0.0,2.521439,5.434791,5.686418,11.998589,7.274513,12.301583,30.312800,2.281858,5.350670,7.754603,4.481865,0.094663,0.198119,0.143246,0.071519,0.283674,3.604362,0.023326,0.147721,0.277050,11.035736,2.053300,9.724237,4.731499,547.30896,41.082893,318.936268,432.264870,3249.895628,28324.407295,4.868346,4.096856,8.121049,13.831080,1.777575,15.435274
min,0.000000,0.000000,0.000000,0.000000,1.000000,2019.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.00000,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,3.000000,2019.000000,4.000000,8.000000,10.000000,0.000000,0.000000,0.000000,237.000000,1.000000,1.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,5.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,1.000000,124.00000,30.870000,0.000000,166.670000,3000.000000,40000.000000,2.000000,1.000000,0.000000,4.000000,2017.000000,26.670000
50%,0.000000,0.000000,1.000000,0.000000,3.000000,2020.000000,7.000000,16.000000,15.000000,0.000000,0.000000,0.000000,528.000000,1.000000,3.000000,4.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,8.000000,9.000000,4.000000,0.070000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,4.000000,0.000000,1.000000,3.00000

In [ ]:
train_df.nunique().sort_values().tail(20)

급지구분_3             13
급지구분_2             14
급지구분_1             14
도로노선방향_3           14
도로노선방향_1           16
주차장유형_노외           16
경상자수               18
주차장유형_노상           20
주차장구분_공영           22
시간                 24
무인CCTV설치한곳의수       25
일                  31
주차기본요금             41
ECLO               46
제한속도               68
설치연도               78
주차구획수             111
주차구획수평균           113
동                 196
ID              39609
dtype: int64

In [ ]:
train_df.nunique().sort_values().head(20)

도시                  1
결제방법_현금, 신용카드       2
결제방법_신용카드           2
결제방법_현금,카드          2
결제방법_교통카드           2
결제방법_신용카드, 교통카드     2
결제방법_현금+신용카드        2
결제방법_무료             3
사고유형                3
사망자수                3
결제방법_카드             3
연                   3
도로형태1               5
노면상태                6
기상상태                6
요일                  7
중상자수                7
구                   8
부상자수                9
도로형태2              10
dtype: int64

In [ ]:
train_df.hist(figsize=(20,16))
plt.show()

In [ ]:
df_missing = train_df.copy()
np.sum(df_missing.isnull())
missing_number = df_missing.isnull().sum().sort_values(ascending=False)
missing_percentage = missing_number/len(df_missing)
missing_info = pd.concat([missing_number,missing_percentage],  axis=1, keys=['missing number','missing percentage'])
missing_info.head(30)

## 2. 상관관계분석

In [ ]:
df_train_corr = train_df.corr().abs().unstack().sort_values(kind="quicksort").reset_index()
df_train_corr.rename(columns={"level_0": "Feature 1", "level_1": "Feature 2", 0: 'Correlation Coefficient'}, inplace=True)
df_train_corr.drop(df_train_corr.iloc[1::2].index, inplace=True)
df_train_corr_nd = df_train_corr.drop(df_train_corr[df_train_corr['Correlation Coefficient'] == 1.0].index)
corr_na =  df_train_corr_nd[~df_train_corr_nd['Correlation Coefficient'].isna()]
corr_na[np.logical_and(corr_na['Feature 1'] != 'ECLO', corr_na['Feature 2'] != 'ECLO')].tail(20)

In [ ]:
for i in train_df.columns.tolist():
  num_by_region = train_df.groupby([i])['ECLO'].mean().sort_values(ascending=False)
  print(f'{i} : {num_by_region}')

In [ ]:
for i in train_df.columns.tolist():
  Max= train_df.groupby([i])['ECLO'].mean().max()
  print(f'{i} : {Max}')
  Min= train_df.groupby([i])['ECLO'].mean().min()
  print(f'{i} : {Min}')
  print('---------------------------------')

# Final Model

### 학습시킬 feature select

In [ ]:
test_x = test_df[['요일', '노면상태', '사고유형','도로형태2', '설치형태_건축물','설치형태_전용주'
                  ,'설치형태_한전주','시간','설치개수','설치개수평균','주차구획수',
                  '관할경찰서명_달성경찰서','CCTV설치여부_N','주차기본요금','설치연도', '제한속도']].copy()
train_x = train_df[['요일', '노면상태', '사고유형','도로형태2', '설치형태_건축물','설치형태_전용주'
                  ,'설치형태_한전주','시간','설치개수','설치개수평균','주차구획수',
                  '관할경찰서명_달성경찰서','CCTV설치여부_N','주차기본요금','설치연도', '제한속도']].copy()
train_y = train['ECLO'].copy()

## 시간 별 칼럼별 ECLO 평균

In [ ]:
def time_range(time):
  if time >= 0 and time <= 7 :
    return 1
  elif time <= 12 :
    return 2
  elif time <= 18 :
    return 3
  else :
    return 4

train_x['time_range'] = train_x['시간'].apply(time_range)
test_x['time_range'] = test_x['시간'].apply(time_range)

- ECLO의 평균이 5 이상인 차대차 이면서 새벽과 늦은저녁인 행은 1
- ECLO가 가장 낮은 오후이면서 차대 사람이면 행은 2
- 나머지는 3으로

In [ ]:
def time_carBycar(data):
  if data['사고유형'] == '차대차':
    if data['time_range'] == 1:
      return 1
    elif data['time_range'] == 4:
      return 1
    else:
      return 2
  elif data['사고유형'] == '차대사람' and data['time_range'] == 3:
    return 0
  else:
    return 2

train_x['time_carBycar_data'] = train_x.apply(time_carBycar, axis = 1)
test_x['time_carBycar_data'] = test_x.apply(time_carBycar, axis = 1)

,요일,노면상태,사고유형,도로형태2,설치형태_건축물,설치형태_전용주,설치형태_한전주,시간,설치개수,설치개수평균,주차구획수,new동,관할경찰서명_달성경찰서,CCTV설치여부_N,주차기본요금,설치연도,제한속도,time_range,time_carBycar_data
0,화요일,건조,차대사람,기타,177.0,30.0,135.0,0,391.0,1.00,500.0,0,0.0,0.0,1136.36,2013.00,8.0,1,2
1,화요일,건조,차대사람,기타,0.0,0.0,0.0,0,932.0,1.00,114.0,0,0.0,0.0,150.00,2015.33,17.5,1,2
2,화요일,건조,차대사람,기타,14.0,31.0,425.0,1,473.0,1.00,0.0,0,0.0,0.0,0.00,2018.25,55.0,1,2
3,화요일,건조,차대차,기타,0.0,0.0,0.0,2,534.0,1.00,374.0,0,0.0,2.0,71.43,2018.30,38.0,1,1
4,화요일,건조,차대차,기타,540.0,57.0,1396.0,4,2057.0,1.03,63.0,0,0.0,0.0,600.00,2018.25,40.0,1,1


In [ ]:
def time_road(data):
  if data['도로형태2'] == '터널':
    return 1
  elif data['도로형태2'] == '고가도로위':
    if data['time_range'] == 1 or data['time_range'] == 4:
      return 1
    else:
      return 0
  elif data['도로형태2'] == '주차장':
    return 2
  elif data['도로형태2'] == '미분류':
    return 2
  else :
    return 0

train_x['time_road_data'] = train_x.apply(time_road, axis = 1)
test_x['time_road_data'] = test_x.apply(time_road, axis = 1)

## 요일별 칼럼별 ECLO 평균

- 요일별 시간

In [ ]:
def sun_time_acci(data):
  if data['요일'] == '일요일':
    if data['time_range'] == 2 or data['time_range'] == 3:
      return 1
    else:
      return 0
  else:
    return 0

train_x['sun_timed_data'] = train_x.apply(sun_time_acci, axis = 1)
test_x['sun_timed_data'] = test_x.apply(sun_time_acci, axis = 1)

- 요일별 사고유형

In [ ]:
def day_carBy(data):
  if data['사고유형'] == '차대차':
    if data['요일'] == '일요일':
      return 1
    elif data['요일'] == '토요일':
      return 1
    else :
      return 0
  elif data['사고유형'] == '차량단독':
    if data['요일'] == '수요일':
      return 2
    else:
      return 0
  elif data['사고유형'] == '차대사람' and data['요일'] == '목요일':
    return 2
  else:
    return 0

train_x['day_carBy_data'] = train_x.apply(day_carBy, axis = 1)
test_x['day_carBy_data'] = test_x.apply(day_carBy, axis = 1)

,요일,노면상태,사고유형,도로형태2,설치형태_건축물,설치형태_전용주,설치형태_한전주,시간,설치개수,설치개수평균,주차구획수,new동,관할경찰서명_달성경찰서,CCTV설치여부_N,주차기본요금,설치연도,제한속도,time_range,time_carBycar_data,time_road_data,sun_timed_data,day_carBy_data
0,화요일,건조,차대사람,기타,177.0,30.0,135.0,0,391.0,1.00,500.0,0,0.0,0.0,1136.36,2013.00,8.0,1,2,0,0,0
1,화요일,건조,차대사람,기타,0.0,0.0,0.0,0,932.0,1.00,114.0,0,0.0,0.0,150.00,2015.33,17.5,1,2,0,0,0
2,화요일,건조,차대사람,기타,14.0,31.0,425.0,1,473.0,1.00,0.0,0,0.0,0.0,0.00,2018.25,55.0,1,2,0,0,0
3,화요일,건조,차대차,기타,0.0,0.0,0.0,2,534.0,1.00,374.0,0,0.0,2.0,71.43,2018.30,38.0,1,1,0,0,0
4,화요일,건조,차대차,기타,540.0,57.0,1396.0,4,2057.0,1.03,63.0,0,0.0,0.0,600.00,2018.25,40.0,1,1,0,0,0


### one-hot encoding

In [ ]:
def one_hot(df):

    # one-hot encoding 실시
    df = pd.get_dummies(df)

    return df

In [ ]:
X_train_eng = one_hot(train_x)
X_test_eng = one_hot(test_x)

# 타겟값은 로그치환 진행
y_train = np.log(train_y)

### train/test data split

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train_eng, y_train , test_size=0.2, random_state=42)

## LGBMRegressor 모델

In [ ]:
# LGBM Regressor 모델 생성
lgbm = LGBMRegressor(random_state=42)

# 탐색할 하이퍼파라미터 그리드 설정
param_grid = {
    'num_leaves': [10, 20, 30],
    'learning_rate': [0.001, 0.01, 0.1],
    'n_estimators': [300, 400, 500, 800, 1200]
}

# GridSearchCV를 사용하여 그리드 탐색 실행
grid_search_lgbm = GridSearchCV(estimator=lgbm, param_grid=param_grid, cv=3, n_jobs=-1)
grid_search_lgbm.fit(X_train, y_train)

# 최적의 하이퍼파라미터와 점수 출력
print("최적의 하이퍼파라미터:", grid_search_lgbm.best_params_)
print("최적의 점수:", grid_search_lgbm.best_score_)

# 최적의 하이퍼파라미터를 적용하여 모델 생성
lgbm_best = LGBMRegressor(**grid_search_lgbm.best_params_,random_state=42)

# 모델 학습
lgbm_best.fit(X_train, y_train)

# 테스트 데이터로 예측
y_train_pred = lgbm_best.predict(X_train)
y_val_pred = lgbm_best.predict(X_val)

# 예측값 로그치환 풀어주기
y_train_pred1 = np.exp(y_train_pred)
y_val_pred1 = np.exp(y_val_pred)

# 타겟값 로그치환 풀어주기
y_train1 = np.exp(y_train)
y_val1 = np.exp(y_val)

# rmsle점수
print(rmsle(y_train1, y_train_pred1))
print(rmsle(y_val1, y_val_pred1))

# 제출 복사하기
sub = submission_df.copy()

# test 예측하기
y_pred_ = lgbm_best.predict(X_test_eng)
y_pred = np.exp(y_pred_)

# 제출파일에 담아주기
sub['ECLO'] = y_pred

# 예측한 그래프
sns.histplot(sub)

## RandomForestRegressor 모델

In [ ]:
# Random Forest Regressor 모델 생성
rf = RandomForestRegressor(random_state=42)

# 탐색할 하이퍼파라미터 그리드 설정
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [8, 10, 12]
}

# GridSearchCV를 사용하여 그리드 탐색 실행
grid_search_rf = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1)
grid_search_rf.fit(X_train, y_train)

# 최적의 하이퍼파라미터와 점수 출력
print("최적의 하이퍼파라미터:", grid_search_rf.best_params_)
print("최적의 점수:", grid_search_rf.best_score_)

# 최적의 하이퍼파라미터를 적용하여 모델 생성
rf_best = RandomForestRegressor(**grid_search_rf.best_params_,random_state=42)

# 모델 학습
rf_best.fit(X_train, y_train)

# 테스트 데이터로 예측
y_train_pred = rf_best.predict(X_train)
y_val_pred = rf_best.predict(X_val)

y_train_pred1 = np.exp(y_train_pred)
y_val_pred1 = np.exp(y_val_pred)

y_train1 = np.exp(y_train)
y_val1 = np.exp(y_val)

print(rmsle(y_train1, y_train_pred1))
print(rmsle(y_val1, y_val_pred1))

sub = submission_df.copy()

y_pred_ = rf_best.predict(X_test_eng)
y_pred = np.exp(y_pred_)

sub['ECLO'] = y_pred

sns.histplot(sub)

## XGBRegressor 모델

In [ ]:
# XGB Regressor 모델 생성
xgb = XGBRegressor(random_state=42)

# 탐색할 하이퍼파라미터 그리드 설정
param_grid = {
    'n_estimators': [300, 400, 500],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.001, 0.01, 0.1]
}

# GridSearchCV를 사용하여 그리드 탐색 실행
grid_search_xgb = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=3, n_jobs=-1)
grid_search_xgb.fit(X_train, y_train)

# 최적의 하이퍼파라미터와 점수 출력
print("최적의 하이퍼파라미터:", grid_search_xgb.best_params_)
print("최적의 점수:", grid_search_xgb.best_score_)

# 최적의 하이퍼파라미터를 적용하여 모델 생성
xgb_best = XGBRegressor(**grid_search_xgb.best_params_,random_state=42)

# 모델 학습
xgb_best.fit(X_train, y_train)

# 테스트 데이터로 예측
y_train_pred = xgb_best.predict(X_train)
y_val_pred = xgb_best.predict(X_val)

y_train_pred1 = np.exp(y_train_pred)
y_val_pred1 = np.exp(y_val_pred)

y_train1 = np.exp(y_train)
y_val1 = np.exp(y_val)

print(rmsle(y_train1, y_train_pred1))
print(rmsle(y_val1, y_val_pred1))

sub = submission_df.copy()

y_pred_ = xgb_best.predict(X_test_eng)
y_pred = np.exp(y_pred_)

sub['ECLO'] = y_pred

sns.histplot(sub)

## GradientBoosting

In [ ]:
model = GradientBoostingRegressor()

# 탐색할 하이퍼파라미터 그리드 정의
param_grid = {
    'n_estimators': [50, 100, 150],  # 트리 개수
    'learning_rate': [0.01, 0.1, 0.2],  # 학습률
    'max_depth': [3, 4, 5]  # 트리 최대 깊이
}

# GridSearchCV를 사용하여 하이퍼파라미터 탐색
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1)
grid_search.fit(X_train, y_train)

gb_best = grid_search.best_estimator_

gb_best.fit(X_train, y_train)

# 테스트 데이터로 예측
y_train_pred = gb_best.predict(X_train)
y_val_pred = gb_best.predict(X_val)

y_train_pred1 = np.exp(y_train_pred)
y_val_pred1 = np.exp(y_val_pred)

y_train1 = np.exp(y_train)
y_val1 = np.exp(y_val)

print(rmsle(y_train1, y_train_pred1))
print(rmsle(y_val1, y_val_pred1))

sub = submission_df.copy()

y_pred_ = gb_best.predict(X_test_eng)
y_pred = np.exp(y_pred_)

sub['ECLO'] = y_pred

sns.histplot(sub)

## Stack

In [ ]:
# 개별 모델 생성
rf = rf_best
lgbm = lgbm_best
xgb = xgb_best
gb = gb_best


# 스태킹 앙상블 모델 생성
estimators = [
    ('rf', rf),
    ('lgbm', lgbm),
    ('xgb', xgb),
    ('gb', gb)
]
stacking_model = StackingRegressor(
    estimators=estimators,
    final_estimator=LinearRegression()
)

# 모델 학습
stacking_model.fit(X_train, y_train)

# 테스트 데이터로 예측
y_train_pred = stacking_model.predict(X_train)
y_val_pred = stacking_model.predict(X_val)

y_train_pred1 = np.exp(y_train_pred)
y_val_pred1 = np.exp(y_val_pred)

y_train1 = np.exp(y_train)
y_val1 = np.exp(y_val)

print(rmsle(y_train1, y_train_pred1))
print(rmsle(y_val1, y_val_pred1))

In [ ]:
if not all(X_train.columns == X_test_eng.columns):
  X_test_eng = X_test_eng[X_train.columns]

In [ ]:
y_pred1 = stacking_model.predict(X_test_eng)
y_pred1

In [ ]:
y_pred = np.exp(y_pred1)
y_pred

In [ ]:
submission_df['ECLO'] = y_pred
submission_df.head()

In [ ]:
sns.histplot(submission_df)